In [1]:
import torch

print(torch.__version__)

1.3.1


# 2.3 自動求梯度
## 2.3.1 概念
上一節介紹的`Tensor`是這個包的核心類，如果將其屬性`.requires_grad`設置為`True`，它將開始追蹤(track)在其上的所有操作。完成計算後，可以調用`.backward()`來完成所有梯度計算。此`Tensor`的梯度將累積到`.grad`屬性中。
> 注意在調用`.backward()`時，如果`Tensor`是標量，則不需要為`backward()`指定任何參數；否則，需要指定一個求導變量。

如果不想要被繼續追蹤，可以調用`.detach()`將其從追蹤記錄中分離出來，這樣就可以防止將來的計算被追蹤。此外，還可以用`with torch.no_grad()`將不想被追蹤的操作代碼塊包裹起來，這種方法在評估模型的時候很常用，因為在評估模型時，我們並不需要計算可訓練參數（`requires_grad=True`）的梯度。

`Function`是另外一個很重要的類。`Tensor`和`Function`互相結合就可以構建一個記錄有整個計算過程的非循環圖。每個`Tensor`都有一個`.grad_fn`屬性，該屬性即創建該`Tensor`的`Function`（除非用戶創建的`Tensor`s時設置了`grad_fn=None`）。

下面通過一些例子來理解這些概念。

## 2.3.2 `Tensor`

In [26]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [27]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


注意x是直接創建的，所以它沒有`grad_fn`, 而y是通過一個加法操作創建的，所以它有一個為`<AddBackward>`的`grad_fn`。

In [28]:
print(x.is_leaf, y.is_leaf)

True False


In [29]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


通过`.requires_grad_()`来用in-place的方式改变`requires_grad`属性：

In [30]:
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)

False
True


## 2.3.3 梯度 

因為`out`是一個標量，所以調用`backward()`時不需要指定求導變量：

In [31]:
out.backward() # 等价于 out.backward(torch.tensor(1.))
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


我們令`out`為 $o$ , 因為
$$
o=\frac14\sum_{i=1}^4z_i=\frac14\sum_{i=1}^43(x_i+2)^2
$$
所以
$$
\frac{\partial{o}}{\partial{x_i}}\bigr\rvert_{x_i=1}=\frac{9}{2}=4.5
$$
所以上面的輸出是正確的。

數學上，如果有一個函數值和自變量都為向量的函數 $\vec{y}=f(\vec{x})$, 那麽 $\vec{y}$ 關於 $\vec{x}$ 的梯度就是一個雅可比矩陣（Jacobian matrix）:

$$
J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)
$$

而``torch.autograd``這個包就是用來計算一些雅克比矩陣的乘積的。例如，如果 $v$ 是一個標量函數的 $l=g\left(\vec{y}\right)$ 的梯度：

$$
v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)
$$

那麽根據鏈式法則我們有 $l$ 關於 $\vec{x}$ 的雅克比矩陣就為:

$$
v \cdot J=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right) \left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)=\left(\begin{array}{ccc}\frac{\partial l}{\partial x_{1}} & \cdots & \frac{\partial l}{\partial x_{n}}\end{array}\right)
$$

注意：grad在反向傳播過程中是累加的(accumulated)，這意味著每一次運行反向傳播，梯度都會累加之前的梯度，所以一般在反向傳播之前需把梯度清零。

In [37]:
# 再來反向傳播一次，注意grad是累加的
print(x)
out2 = x.sum()
print(out2)
out2.backward()
print(x.grad)

out3 = x.sum()
print(out3)
x.grad.data.zero_()  # Look here!
print(out3)
out3.backward()
print(x.grad)

tensor([100.], requires_grad=True)
tensor(100., grad_fn=<SumBackward0>)
tensor([3.])
tensor(100., grad_fn=<SumBackward0>)
tensor(100., grad_fn=<SumBackward0>)
tensor([1.])


In [22]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


現在 `y` 不是一個標量，所以在調用`backward`時需要傳入一個和`y`同形的權重向量進行加權求和得到一個標量。。

In [23]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)

print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


再来看看中断梯度追踪的例子：

In [24]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
    
print(x, x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

tensor(1., requires_grad=True) True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [25]:
y3.backward()
print(x.grad)

tensor(2.)


為什麽是2呢？$ y_3 = y_1 + y_2 = x^2 + x^3$，當 $x=1$ 時 $\frac {dy_3} {dx}$ 不應該是5嗎？事實上，由於 $y_2$ 的定義是被`torch.no_grad():`包裹的，所以與 $y_2$ 有關的梯度是不會回傳的，只有與 $y_1$ 有關的梯度才會回傳，即 $x^2$ 對 $x$ 的梯度。

上面提到，`y2.requires_grad=False`，所以不能調用 `y2.backward()`

In [38]:
# y2.backward() # 会报错 RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

如果我們想要修改`tensor`的數值，但是又不希望被`autograd`記錄（即不會影響反向傳播），那麽我麽可以對`tensor.data`進行操作.

In [36]:
x = torch.ones(1,requires_grad=True)

print(x.data) # 還是一個tensor
print(x.data.requires_grad) # 但是已經是獨立於計算圖之外

y = 2 * x
x.data *= 100 # 只改變了值，不會記錄在計算圖，所以不會影響梯度傳播

y.backward()
print(x) # 更改data的值也會影響tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
